# Example KubeHound queries 

This file contains examples queries for kubehound. This file should be mostly read only. No modification will be ported to the original file, it will be kept only for the duration of the docker container.

Note: You may need to adjust the value on the cell below and execute (ctrl + enter) before running any other cells.

## Initial Setup

Connect to the kubegraph server by running the cell below

In [ ]:
%%graph_notebook_config
{
  "host": "host.docker.internal",
  "port": 8182,
  "ssl": false,
  "gremlin": {
    "traversal_source": "g",
    "username": "",
    "password": "",
    "message_serializer": "graphsonv3"
  }
}

Now set the appearance customizations for the notebook. You can see a guide on possible options [here](https://github.com/aws/graph-notebook/blob/623d43827f798c33125219e8f45ad1b6e5b29513/src/graph_notebook/notebooks/01-Neptune-Database/02-Visualization/Grouping-and-Appearance-Customization-Gremlin.ipynb#L680)

In [ ]:
%%graph_notebook_vis_options
{
  "edges": {
    "smooth": {
      "enabled": true,
      "type": "dynamic"
    },
    "arrows": {
      "to": {
        "enabled": true,
        "type": "arrow"
      }
    }
  }
}

## Critical Path Queries

### All critical paths from external services

Critical paths from external services can highlight most likely paths an attacker can take following the compromise of an external service. In a well-configured cluster, such paths should be limited to very few (if any) services.

**NOTE** an `ENDPOINT_EXPLOIT` edge does not signal that the endpoint is *necessarily* exploitable but serves as a useful starting point for path traversal queries


In [ ]:
%%gremlin -d class -g critical -le 50 -p inv,oute
kh.services().
    criticalPaths().
    by(elementMap()).
    limit(100)  // Limit the number of results for large clusters

### All critical paths from groups

Critical paths from groups can highlight overprivileged groups and help quickly assess the impact of compromised credentials.

In [ ]:
%%gremlin -d class -g critical -le 50 -p inv,oute
kh.groups().
    criticalPaths().
    by(elementMap()).
    limit(100)  // Limit the number of results for large clusters

### Complex critical path queries
The [KubeHound DSL](https://kubehound.io/queries/dsl/) is fully compatible with the Gremlin language, so you can use any gremlin function in your queries to add additional filters if needed. The example below attempts to filter for Kubernetes services exposed on port 443 attached to `elasticsearch` containers, and limits attack paths to 6 hops or less.

In [ ]:
%%gremlin -d class -g critical -le 50 -p inv,oute
kh.services().
    has("port", 443).   // Look for exposed port on 443 only
    not(has("namespace", within("system", "kube"))).    // Exclude endpoints from the 'kube' and 'system' namespaces
    where(__.out().hasLabel("Container").has("image", TextP.containing("elasticsearch"))).  // Only accept endpoints attached to elasticsearch containers
    criticalPaths(6).   // Limit to critical paths of 6 hops or less
    by(elementMap()).
    limit(100)  // Limit the number of results for large clusters

See the [documentation](https://kubehound.io/queries/dsl/#criticalpaths-step) for further examples of critical path analysis